<a href="https://colab.research.google.com/github/yeong95/LP_Recommendation/blob/master/SASRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import module

In [ ]:
%tensorflow_version 1.12

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.12`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [ ]:
import sys
import pandas as pd
import numpy as np
sys.version

'3.7.10 (default, Feb 20 2021, 21:17:23) \n[GCC 7.5.0]'

In [ ]:
sys.path.insert(0, "/content/drive/My Drive/Sequential_Recommendation/models/SASRec")
sys.path.insert(0, "/content/drive/My Drive/Sequential_Recommendation")

In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import argparse
sys.path.append("..")

os.environ["CUDA_VISIBLE_DEVICES"]='0'   # 원래 코드는 1 
import tensorflow as tf
import numpy as np

from model import Model
from make_datasets_SASRec import make_datasets
from DataInput_SASRec import DataIterator
from evaluation import SortItemsbyScore,Metric_HR,Metric_MRR
from modules import *
from util import *
import time

#lpoint SASRec

In [ ]:
def parse_args():
    parser = argparse.ArgumentParser(description='SASRec')
    parser.add_argument('--dataset', default='lpoint', type=str)
    parser.add_argument('--train_dir', default='train', type=str)
    parser.add_argument('--batch_size', default=128, type=int)
    parser.add_argument('--lr', default=0.001, type=float)
    parser.add_argument('--max_len', default=50, type=int)
    parser.add_argument('--maxlen', default=50, type=int)
    parser.add_argument('--hidden_units', default=50, type=int)
    parser.add_argument('--num_blocks', default=2, type=int)
    parser.add_argument('--num_epochs', default=201, type=int)
    parser.add_argument('--num_heads', default=1, type=int)
    parser.add_argument('--dropout_rate', default=0.5, type=float)
    parser.add_argument('--l2_emb', default=0.0, type=float)
    return parser.parse_args(args=[])


In [ ]:
args = parse_args()

In [ ]:
file_path = '/content/drive/MyDrive/Sequential_Recommendation/datasets/제6회 L.POINT Big Data Competition-분석용데이터-02.거래 정보.csv'
data = pd.read_csv(file_path, encoding='utf-8')

In [ ]:
data.head()

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
0,21922,104999,1,A03,unknown,20190920,12:41,5990,1
1,21279,104907,4,A03,unknown,20190920,10:27,10900,1
2,39423,105124,11,A03,unknown,20190920,17:26,12900,1
3,18362,104010,1,A03,unknown,20190920,09:57,9900,1
4,39423,105124,13,A03,0565,20190920,17:26,2990,1


In [ ]:
removed_data = data[~(data.pd_c == 'unknown')]
removed_data

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
4,39423,105124,13,A03,0565,20190920,17:26,2990,1
5,21279,104907,1,A03,0565,20190920,10:27,2990,1
6,48969,104882,5,A03,0572,20190920,10:05,7490,1
7,30533,103245,5,A03,0670,20190920,14:36,49800,1
8,64346,104317,1,A03,0543,20190920,22:34,26900,1
...,...,...,...,...,...,...,...,...,...
599956,35311,2839,3,B03,0339,20190927,14:09,3500,1
599957,35311,2839,4,B03,0339,20190927,14:09,3600,2
599958,35311,2839,5,B03,0339,20190927,14:09,3500,1
599959,35311,2839,2,B03,0339,20190927,14:09,4500,1


In [ ]:
removed_data = removed_data[~(removed_data.buy_ct == 0)]

In [ ]:
removed_data = removed_data.sort_values(by=['trans_id','trans_seq'])

In [ ]:
removed_data.head()

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
478560,47617,1,1,B01,0897,20190702,21:19,20640,3
478481,47617,1,2,B01,0548,20190702,21:19,14980,1
478503,47617,1,3,B01,0894,20190702,21:19,25900,1
478521,47617,1,4,B01,1447,20190702,21:19,5490,1
478543,47617,1,5,B01,1529,20190702,21:19,5900,1


In [ ]:
removed_data = removed_data.astype({'pd_c':'int'})

In [ ]:
removed_data.head()

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
478560,47617,1,1,B01,897,20190702,21:19,20640,3
478481,47617,1,2,B01,548,20190702,21:19,14980,1
478503,47617,1,3,B01,894,20190702,21:19,25900,1
478521,47617,1,4,B01,1447,20190702,21:19,5490,1
478543,47617,1,5,B01,1529,20190702,21:19,5900,1


In [ ]:
new_data = removed_data[['trans_id','trans_seq', 'pd_c']]

In [ ]:
new_data.rename(columns={'trans_id':'user','trans_seq':'timestamps','pd_c':'item'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
new_data.head()

,user,timestamps,item
478560,1,1,897
478481,1,2,548
478503,1,3,894
478521,1,4,1447
478543,1,5,1529


In [ ]:
item_dataset = make_datasets(new_data, args.max_len)
d_train, d_test, d_info = item_dataset

In [ ]:
d_train.head()

,user,seq,target
0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
d_test.head()

,user,seq,target
0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[6]
1,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[3]
2,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[14]
3,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[6]
4,7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[19]


In [ ]:
d_train[d_train.user == 100].seq.values

array([array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 176,
         4,  37,  61, 214, 214,  95,  96, 207, 215, 216,  68], dtype=int32)],
      dtype=object)

In [ ]:
d_train[d_train.user == 100].target.values

array([array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   4,
        37,  61, 214, 214,  95,  96, 207, 215, 216,  68, 213], dtype=int32)],
      dtype=object)

In [ ]:
d_test[d_test.user == 100].seq.values

array([array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 176,   4,
        37,  61, 214, 214,  95,  96, 207, 215, 216,  68, 213], dtype=int32)],
      dtype=object)

In [ ]:
d_test[d_test.user == 100].target.values

array([list([93])], dtype=object)

In [ ]:
len(d_train)

46574

In [ ]:
num_usr, num_item, items_usr_clicked, _, _ = d_info
all_items = [i for i in range(num_item)]

In [ ]:
print(num_usr)
print(num_item)

46574
1384


In [ ]:
# remove session that target item is in the training sequence 
user_tmp_list = []
for user in items_usr_clicked.keys():
  target_item = items_usr_clicked[user][-1]
  for train_item in items_usr_clicked[user][:-1]:
    if target_item == train_item:
      user_tmp_list.append(user)
      break


In [ ]:
len(user_tmp_list)

7813

In [ ]:
for user in user_tmp_list:
  items_usr_clicked.pop(user)

In [ ]:
print(len(items_usr_clicked))
print(46574-7813)

38760
38761


In [ ]:
# Define DataIterator
trainIterator = DataIterator('train',d_train, args.batch_size, args.max_len,
                              all_items, items_usr_clicked, shuffle=True)

In [ ]:
# Define Model
model = Model(usernum=num_usr,
                itemnum=num_item,
                args=args)


Instructions for updating:
Use `tf.cast` instead.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.








In [ ]:
if not os.path.isdir(args.dataset + '_' + args.train_dir):
    os.makedirs(args.dataset + '_' + args.train_dir)
with open(os.path.join(args.dataset + '_' + args.train_dir, 'args.txt'), 'w') as f:
    f.write('\n'.join([str(k) + ',' + str(v) for k, v in sorted(vars(args).items(), key=lambda x: x[0])]))
f.close()

In [ ]:
f = open(os.path.join(args.dataset + '_' + args.train_dir, 'SASRec_log.txt'), 'w')
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.allow_soft_placement = True

In [ ]:
T = 0.0
t0 = time.time()

In [ ]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    

    for epoch in range(1, args.num_epochs):

        #train
        cost_list = []
        for train_input in tqdm(trainIterator,desc='epoch {}'.format(epoch),total=trainIterator.total_batch):
            batch_usr, batch_seq, batch_pos, batch_neg = train_input            
            auc, cost, _ = sess.run([model.auc, model.loss, model.train_op],
                                {model.u: batch_usr, model.input_seq: batch_seq,  
                                   model.pos: batch_pos, model.neg: batch_neg, model.is_training: True})
            cost_list.append(cost)
        mean_cost = np.mean(cost_list)
        
        #saver.save(sess, FLAGS.save_path)

        if epoch % 20 == 0:
          t1 = time.time() - t0
          T += t1        
          t_test = all_item_evaluate(model, item_dataset, all_items, args, sess)
          print('epoch:%d, time: %f(s), test (NDCG@10: %.4f, HR@10: %.4f)' % ( epoch, T, t_test[0], t_test[1]))
          f.write(str(epoch)+'epoch:'+' ' + '(NDCG@10,HR@10)' + ' ' + str(t_test) + '\n')
          f.flush()

          t0 = time.time()
        

epoch 21:   1%|▏         | 5/364 [00:00<00:08, 44.72it/s]

epoch:20, time: 190.179037(s), test (NDCG@10: 0.1715, HR@10: 0.3015)


epoch 41:   1%|▏         | 5/364 [00:00<00:08, 40.81it/s]

epoch:40, time: 362.332646(s), test (NDCG@10: 0.1754, HR@10: 0.3049)


epoch 61:   1%|▏         | 5/364 [00:00<00:08, 43.74it/s]

...

epoch:160, time: 1399.066607(s), test (NDCG@10: 0.1856, HR@10: 0.3167)


epoch 181:   1%|          | 4/364 [00:00<00:09, 39.23it/s]

epoch:180, time: 1571.446790(s), test (NDCG@10: 0.1850, HR@10: 0.3146)


epoch 200: 100%|██████████| 364/364 [00:08<00:00, 42.84it/s]


epoch:200, time: 1742.054423(s), test (NDCG@10: 0.1773, HR@10: 0.3103)


In [ ]:
batch_usr

array([  4819,   1061,  76527,  78922,  26879,  38777,  15976,  56399,
       109886,  42107,  24007, 102748,  90839,  82646,  53069,  43444,
        67838,  88853,  10345,  87741,  92463,  66604,  54500,  86767,
       104909, 103707,  40712,  11109,  73064,  26502,  71835,  14075,
        26852,  49849,  35503,  62348, 111185,  99166,  90396,  84297,
        52458,  50486,  58252,  29334,   8247,  44240,  82529,  88328,
        76797,  67200,  62377,  34225,  31159,  13269,   5075,  65774,
        79828,  36474,  38235, 114382,  36589,  51502,  99397, 100336,
        85127,  60527,  90324,  75689,  81226,  19972,  48599,  32874,
        44374, 101198, 113054,  81808,  82387,  81134,  61882, 113126,
         5288,  14371,  13044,  44941,  91902,  28396, 107574,  13146,
        95276,  54967,  58770,  52452,  60449,  92173,  38456,  63059,
       108067,  62416,  52797,  99098,  12962,   7047,  26202,  79382,
        53387, 113231, 108219, 110014])

In [ ]:
batch_seq[-1]

In [ ]:
batch_pos[-1]

In [ ]:
batch_neg[-1]  

# Real

In [ ]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    

    for epoch in range(1, args.num_epochs):

        #train
        cost_list = []
        for train_input in tqdm(trainIterator,desc='epoch {}'.format(epoch),total=trainIterator.total_batch):
            batch_usr, batch_seq, batch_pos, batch_neg = train_input            
            auc, cost, _ = sess.run([model.auc, model.loss, model.train_op],
                                {model.u: batch_usr, model.input_seq: batch_seq,  
                                   model.pos: batch_pos, model.neg: batch_neg, model.is_training: True})
            cost_list.append(cost)
        mean_cost = np.mean(cost_list)
        
        #saver.save(sess, FLAGS.save_path)

        if epoch % 20 == 0:
          t1 = time.time() - t0
          T += t1        
          t_test = all_item_evaluate(model, item_dataset, all_items, args, sess)
          print('epoch:%d, time: %f(s), test (NDCG@10: %.4f, HR@10: %.4f)' % ( epoch, T, t_test[0], t_test[1]))
          f.write(str(epoch)+'epoch:'+' ' + '(NDCG@10,HR@10)' + ' ' + str(t_test) + '\n')
          f.flush()

          t0 = time.time()
        

epoch 20: 100%|██████████| 364/364 [00:08<00:00, 42.38it/s]


.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


epoch 21:   1%|          | 4/364 [00:00<00:09, 38.91it/s]

epoch:20, time: 176.093315(s), test (NDCG@10: 0.1697, HR@10: 0.2947)


epoch 40: 100%|██████████| 364/364 [00:08<00:00, 41.77it/s]


...

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


epoch 181:   1%|▏         | 5/364 [00:00<00:08, 43.13it/s]

epoch:180, time: 1565.825170(s), test (NDCG@10: 0.1839, HR@10: 0.3159)


epoch 200: 100%|██████████| 364/364 [00:08<00:00, 41.45it/s]


.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
epoch:200, time: 1738.904731(s), test (NDCG@10: 0.1783, HR@10: 0.3099)
